In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()
ps.set_option('compute.ops_on_diff_frames', True)

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
# Loading csv data into pyspark DataFrame
customers=spark.read.options(header=True,inferSchema=True).csv('C:\Users\avitr\Downloads\customer.csv')
transactions=spark.read.options(header=True,inferSchema=True).csv('C:\Users\avitr\Downloads\transactions.csv')

In [ ]:
# Converting pyspark DataFrame to pandas DataFrame
customers_df=customers.pandas_api()
transactions_df=transactions.pandas_api()

In [ ]:
customers_df.head()

In [ ]:
transactions_df.head()

If the cell size is larger than 5, hide the customer's postcode for each combination of state, gender, and age

In [ ]:
# customers_df[col].str.len() returns the length of the values as a Series
# length of state, gender and age combined should be larger than 5
# data obeying it is obtained using the condition customers_df['state'].str.len()+customers_df['gender'].str.len()+customers_df['age'].astype('str').str.len()>5
# everytime the condition is satisfied, postcode is replaced with ****** using df.mask
# mask replaces a value when the given confition is satisfied
customers_df.postcode=customers_df.postcode.mask(customers_df['state'].str.len()+customers_df['gender'].str.len()+customers_df['age'].astype('str').str.len()>5,'******')

In [ ]:
customers_df.head()

Apply bucketing for the age column using the ranges [20-24], [25-29], and [30-34] for 5 years. Ages of 22 and 35, respectively, become [20–24] and [35–39]. Bucketing includes everyone.

In [ ]:
# age falls in the range [(age//5)*5,(age//5)*5+4]
customers_df.age='['+((customers_df.age//5)*5).astype('int').astype('str')+'-'+((customers_df.age//5)*5+4).astype('int').astype('str')+']'

In [ ]:
customers_df.head()

correct or add loyalty flag (column loyal_customer, values true/false)
\
using following conditions: if customer transacted on the amount greater
\
than 1000 per month loyal_customer = true, otherwise false Note: each 
\
transaction total amount is in column Total

In [ ]:
# to access datetime features, convert Date column to datetime object using to_datetime()
dates=ps.to_datetime(transactions_df.Date,format='%m/%d/%Y')

In [ ]:
# groupby CustomerID gives all the money spent by that customer
# groupby Year after CustomerID gives all the money spent by that customer in every year
# groupby Month after CustomerID, Year gives all the money spent by that customer in every month of every year
# sum of the Total money spent in a month should be greater than 1000 for all months
# it is for every month using transactions_df.groupby(['CustomerID',dates.dt.year.rename('Year'),dates.dt.month.rename('Month')]).Total.sum()>1000
# to check if the condition is obeyed for all months,
# first apply reset_index() to access the grouped values as a DataFrame
# now the boolen indicating if the spend is greater than 1000 for each month should be True for all months
# it is ensured by first grouping by CustomerID and applying .all()
# a Series of booleans indexed by CustomerID is obtained
loyal_customer=((transactions_df.groupby(['CustomerID',dates.dt.year.rename('Year'),dates.dt.month.rename('Month')]).Total.sum()>1000).reset_index()).groupby('CustomerID').Total.all().rename('loyal_customer')

In [ ]:
# customer_df contains loyal_customer but it is wrong
# drop it
# join the customer_df with the loyal_customer values obtained
customers_df=customers_df.drop('loyal_customer',axis=1).join(loyal_customer,on='person_id')

In [ ]:
customers_df.head()

filter out transactions of customers that do not present in the customer
dataset after previous treatments.

In [ ]:
# right join transactions_df with customers_df.person_id
# this way, CustomerID values not in customers_df are eliminated
# drop the person_id column as it is not required
transactions_df=transactions_df.join(customers_df.person_id,how='right').drop('person_id',axis=1)

In [ ]:
transactions_df

Provide summary of spendings by days of week: how much customers
spend on Mondays, Tuesdays ...Sundays

In [ ]:
# groupby day of the week and describe
transactions_df.groupby([dates.dt.dayofweek]).Total.describe()

For transactions happening on Wednesdays cap the Total amount to 99
if the amount is more or equal to 100 (what do you do to the Tax, Unit price
and Quantity columns in this case?)

In [ ]:
# if total>=100 on wednesdays, total is changed to 99
# wednesday is ensured using dates.dt.day_of_week==2
# total is ensured using transactions_df.Total>=100
# unit_price becomes 99/(1.05*quantity)
transactions_df['Unit price']=transactions_df['Unit price'].mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),99/(1.05*transactions_df['Quantity']))
# cogs is unit_price*quantity
transactions_df.cogs=transactions_df.cogs.mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),transactions_df['Quantity']*transactions_df['Unit price'])
# tax is 5% of cogs
transactions_df['Tax 5%']=transactions_df['Tax 5%'].mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),transactions_df.cogs*0.05)
# gross income is tax
transactions_df['gross income']=transactions_df['Tax 5%'].round(4)
transactions_df.Total=transactions_df.Total.mask((dates.dt.day_of_week==2)&(transactions_df.Total>=100),99)

In [ ]:
transactions_df.head(10)

For column Time in transactions round down by 15 min periods.
Example: 19:21 - becomes 19:15, 17:36 - becomes 17:30, 10:12 - becomes
10:00, 12:50 - becomes 12:45

In [ ]:
# minutes in time can be obtained from 3rd index
# transactions_df.Time.str[3:]
# ((transactions_df.Time.str[3:]).astype('int')//15)*15 gives the rounded down value of minutes
# this is converted to string
# we have 0,15,30,45 as possible values
# but 0 is represented as 00 in minutes
# it is achieved using .replace('0','00')
# rounded down minutes is added hours and assigned to Time
transactions_df.Time=transactions_df.Time.str[:3]+(((transactions_df.Time.str[3:]).astype('int')//15)*15).astype('str').replace('0','00')

In [ ]:
transactions_df.head()

In [ ]:
# saving data
customers_df=customers_df.to_spark()
customers_df.toPandas().to_csv('customer_final.csv')
transactions_df=transactions_df.to_spark()
transactions_df.toPandas().to_csv('transactions_final.csv')

# aa

In [4]:
import pandas as pd
s = pd.Series(range(5))

In [5]:
s

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [6]:
s.where(s > 1, 10)

0    10
1    10
2     2
3     3
4     4
dtype: int64

In [7]:
s.mask(s > 1, 10)

0     0
1     1
2    10
3    10
4    10
dtype: int64

In [8]:
s

0    0
1    1
2    2
3    3
4    4
dtype: int64